# Hyperparameter?
* 정의 모델의 성능에 영향을 끼치지만 자동으로 최적화되지 않는 파라미터
* 사용자가 실험을 통해 직텁 튜닝해야함 (AutoML이 이 것을 자동으로 해줌)
* 경험적(emphricial) or heuristic한 방법을 통해 찾아야함
### 하이퍼 파라미터 종류
* learning rate, mini-batch
* 신경망의 깊이, 너비 등 신경망의 크기(AutoML에서도 이를 위한 NAS라는 연구주제가 있음)
* ReLU or LeakyReLU중 어떤 activation을 사용? (사소함)

### 결론
* 하이퍼파라미터 튜닝은 모델의 학습이 종료된 후에 이루어짐
* 모델이 클 수록 업데이트가 느림
* 따라서 중요한 또는 사소한 하이퍼 파라미터가 무엇인지 구분할 수 있어야함
* 성능에 큰 영향을 끼치는 하이퍼파라미터 위주로 먼저 튜닝하여 최고의 성능을 이끌어낼 수 있어야 함
* 이를 위해 많은 실정 경험 필요

# 효율적인 연구/개발 진행 방법
* 오픈된 모델, 코드를 그대로 가져오더라도 우리의 문제에 완벽 하게 동장하지 않음
* 따라서 계속하여 성능을 개선하는 업무를 해야 함
## 그럼 어떻게 해야할까?
1. basecamp를 세우기 즉 베이스 라인 구축
* 최소한의 가정과 가장 간단한 방법으로 베이스라인 구축하여 여기서부터 차근차근 개선해 나가는 것이 중요
* 최신의 멋진 논문의 내용을 적용해보고 싶을 수 있겠지만 작동하지 않으면 시간을 낭비하고 무용지물이 됨(모든 논문들은 자신의 방법이 최고라고 자랑하지만, 대부분의 논문은 널리 사용되지 않음)
* 따라서 가장 간단한 방법으로 먼저 뒤를 든든하게 만들고, 이후에 공격적인 방법들을 차근차근 적용해볼 수 있을 것임
* 그러면 나중에 모든 추가적인 노력이 실패하더라도, 최소한의 성능은 보장될 수 있음

## 하이퍼 파라미터에 따른 결과물 정리
* 하이퍼파라미터(hp)는 많은 실험을 반복 수행하며 성능 체크가 필요
* 실험마다 Loss, Accuracy, weight parameter 파일 등 관리할 내용이 너무 많은 것이 문제
* **따라서 실험 결과들을 체계적으로 저장하고 관리할 수 있는 능력과 시스템을 갖춰야 함**
* MLFlow, WanDB를 적극 사용하면 실험 결과를 잘 관리할 수 있음

## 파이프라인 구축
* 연구/실험 과정에 대한 파이프라인 구축</br>
![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-11/02-workflow.png)
* 반복되는 작업을 최대한 자동화할 수 있도록 하여 연구를 효율적으로 진행함
* 머신러닝 업무를 진행하면서 업무 프로세스를 잘 정리하고, 그 중에서 반복되는 작업들이 어떻게 구성되어 있는지를 판단해야 함
1. 업무 프로세스 정리
2. 반복되는 작업 구성 판단
3. 자동화가 가능한 부분을 script로 구현하여 파이프라인을 점차 구성해나감
* (위 작업을 자동화하기위한 workflow가 있음(Airflow, Kubeflow))
</br>
4. 자동화 파이프라인을 구축하는 과정에서 사내의 요구사항을 모아 공개된 도구들을 활용하여 다 함께 구축하는 것이 필요함

# 적응형 학습률
* Learning rate: 핵심 파라미터. 가장 먼저 튜냉하야함
* 너무 적음: local minima에 빠짐, 너무 큼: loss surface의 골짜기가 너무 가파름
* e.g. 처음에 큰 학습률을 가져가되, 현재 epoch의 손실 값이 이전 epoch의 손실 값보다 나아지지 않을 경우, 학습률을 일정 비율로 줄임
* 일정 비율은 learning rate decay ratio, 0.5 ~ 0.1 값을 활용
* 적응형 학습률에 대한 가장 많이 쓰이는 알고리즘은 **Adam**임
![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-11/03-adaptive_lr_lineage.png)

## 모멘텀
![](https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-11/03-momentum_example.png)
이점:
1. local minima를 쉽게 탈출
2. 학습 속도 가속화
**예시**
* 위 그림에서오른쪽으로 진행하는 와중에 진동이 심할 수 있음
* 모멘텀을 적용해 진동을 상쇄하면서 오른쪽으로 가는 속도가 커짐 -> 학습 최적화 빨라짐

## 적응형 학습률(Adaptive learning rate)
* learning rate를 튜닝하기 위해 쓰임
* 학습률이 자동으로 정해지게 하기위함

#### 학습률 스케줄링(Learning rate schedulling)
1. initial learning rate로 모델 최적화 수행
2. 일정 epoch 또는 iteration이 경과한 후에 decay 시작 또는 해당 epoch에서 모델의 학습이 더이상 진정되지 않을 때 decay 시작
3. learning rate decay는 linear적으로 적용될 수도 있으며 비율(ratio)이 곱해지는 형태로 적용되거나 코사인 함수의 형태로 적용될 수도 있음(감쇠되는 형태 또는 비율 또한 HP임)

### Adam optimizer
* 가장 널리 쓰임
* 기존의 적응형 학습률 방식에 모멘텀(momentum)이 추가됨
* 그 외 기존 알고리즘을 보완하여 가장 보편적으로 쓰이는 알고리즘이 됨
* 따라서 대부분의 문제에서 좋은 성능을 얻음
* 모델이나 학습 방법(training scheme)이 learning rate에 강인(robust)해지는 효과를 얻을 수 있음
* 따라서 Adam의 기본 셋팅을 가지고 일단 문제 접근을 시작하는 것이 좋음

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [2]:
california = fetch_california_housing()

df = pd.DataFrame(california.data, columns=california.feature_names)
df["Target"] = california.target
df.tail()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847
20639,2.3886,16.0,5.254717,1.162264,1387.0,2.616981,39.37,-121.24,0.894


In [3]:
scaler = StandardScaler()
scaler.fit(df.values[:, :-1])
df.values[:, :-1] = scaler.transform(df.values[:, :-1])

df.tail()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Target
20635,-1.216128,-0.289187,-0.155023,0.077354,-0.512592,-0.049110,1.801647,-0.758826,0.781
20636,-0.691593,-0.845393,0.276881,0.462365,-0.944405,0.005021,1.806329,-0.818722,0.771
20637,-1.142593,-0.924851,-0.090318,0.049414,-0.369537,-0.071735,1.778237,-0.823713,0.923
20638,-1.054583,-0.845393,-0.040211,0.158778,-0.604429,-0.091225,1.778237,-0.873626,0.847
20639,-0.780129,-1.004309,-0.070443,0.138403,-0.033977,-0.043682,1.750146,-0.833696,0.894


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

c:\Users\PiusHwang\Anaconda3\envs\aicomp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = torch.from_numpy(df.values).float()

x = data[:, :-1]
y = data[:, -1:]

print(x.shape, y.shape)

torch.Size([20640, 8]) torch.Size([20640, 1])


In [6]:
n_epochs = 4000
batch_size = 256
print_interval = 200

In [7]:
model = nn.Sequential(
    nn.Linear(x.size(-1), 6),
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 4),
    nn.LeakyReLU(),
    nn.Linear(4, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y.size(-1))
)

In [8]:
optimizer = optim.Adam(model.parameters())

In [20]:
for it in range(n_epochs):
    indices = torch.randperm(x.size(0))
    x_ = torch.index_select(x, dim=0, index = indices)
    y_ = torch.index_select(y, dim = 0, index = indices)
    x_ = x_.split(batch_size, dim = 0)
    y_ = y_.split(batch_size, dim = 0)

    y_hat = []
    total_loss = 0

    for x_i, y_i in zip(x_, y_):
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        total_loss+=float(loss)
        y_hat+= [y_hat_i]
    
    total_loss = total_loss/len(x_)
    if (it + 1) % print_interval == 0:
        print('Epoch %d: loss = %.4e' % (it + 1, total_loss))

y_hat = torch.cat(y_hat, dim = 0)
y = torch.cat(y_, dim = 0)


Epoch 200: loss = 3.4949e-01
Epoch 400: loss = 3.3559e-01
Epoch 600: loss = 3.2999e-01
Epoch 800: loss = 3.2592e-01
Epoch 1000: loss = 3.2374e-01
Epoch 1200: loss = 3.2400e-01
Epoch 1400: loss = 3.2340e-01
Epoch 1600: loss = 3.2032e-01
Epoch 1800: loss = 3.1349e-01
Epoch 2000: loss = 3.0009e-01
Epoch 2200: loss = 2.9762e-01
Epoch 2400: loss = 2.9550e-01
Epoch 2600: loss = 2.9394e-01
Epoch 2800: loss = 2.9386e-01
Epoch 3000: loss = 2.9234e-01
Epoch 3200: loss = 2.9318e-01
Epoch 3400: loss = 2.9190e-01
Epoch 3600: loss = 2.9211e-01
Epoch 3800: loss = 2.9201e-01
Epoch 4000: loss = 2.9132e-01


In [ ]:
df = pd.DataFrame([y, y_hat], dim=1).detach().numpy()
            